### Importaciones de paquetes necesarios

In [13]:
from deap import creator, base, tools, algorithms

import random
import numpy as np
import pandas as pd

### Creación del turno de enfermería

In [32]:
from turnosenfermeria import TurnosEnfermeria

In [33]:
preferencias = pd.read_csv("./data/datos_enfermeria.txt", sep=' ')
print (preferencias)

   Nombre  Pref_M  Pref_T  Pref_N
0   Enf_A       1       1       0
1   Enf_B       1       0       1
2   Enf_C       1       1       1
3   Enf_D       0       1       0
4   Enf_E       0       0       1
5   Enf_F       0       1       1
6   Enf_G       1       1       0
7   Enf_H       0       0       1
8   Enf_I       1       1       1
9   Enf_J       1       1       0
10  Enf_K       1       1       1
11  Enf_L       1       1       0
12  Enf_M       1       0       1


In [34]:
enfermeras = preferencias["Nombre"]
print (enfermeras)

0     Enf_A
1     Enf_B
2     Enf_C
3     Enf_D
4     Enf_E
5     Enf_F
6     Enf_G
7     Enf_H
8     Enf_I
9     Enf_J
10    Enf_K
11    Enf_L
12    Enf_M
Name: Nombre, dtype: object


In [36]:
turnos = TurnosEnfermeria (enfermeras, preferencias)


# Algoritmo Genético Simple

In [44]:
tamano_individuo = 21
tamano_poblacion = len (preferencias)
max_generaciones = 20

toolbox = base.Toolbox()

In [51]:
# Establecemos cual va a ser la función de ajuste e inicializamos los individuos y la población

creator.create("ClaseAjusteMin", base.Fitness, weights=(-1.0,))
creator.create("ClaseIndividuo", list, fitness=creator.ClaseAjusteMin)

toolbox.register("ceroOrUno", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.ClaseIndividuo, toolbox.ceroOrUno, tamano_individuo)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, tamano_poblacion)
poblacion = toolbox.population()

toolbox.register("evaluate", turnos.getCoste)